In [23]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [25]:
fashion_mnist=keras.datasets.fashion_mnist

In [26]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [27]:
train_images=train_images/255.0
test_images=test_images/255.0

In [28]:
train_images[0].shape

(28, 28)

In [29]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [30]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])),
                                               loss='sparse_categorical_crossentropy',
                                               metrics=['accuracy'])
  
  return model

In [31]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [32]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,
                          directory='output',
                          project_name='Mnist_Fashion')

In [33]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9113333225250244

Best val_accuracy So Far: 0.9133333563804626
Total elapsed time: 00h 03m 15s


In [34]:
model=tuner_search.get_best_models(num_models=1)[0]

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        153648    
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 64)                1486912   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,642,490
Trainable params: 1,642,490
Non-trainable params: 0
_________________________________________________________________


In [36]:
#Retrain with the obtained

model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1547 - accuracy: 0.9422 - val_loss: 0.2744 - val_accuracy: 0.9093
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1149 - accuracy: 0.9575 - val_loss: 0.2728 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0842 - accuracy: 0.9687 - val_loss: 0.3281 - val_accuracy: 0.9040
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0639 - accuracy: 0.9770 - val_loss: 0.3728 - val_accuracy: 0.9092
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0485 - accuracy: 0.9818 - val_loss: 0.4854 - val_accuracy: 0.9010
Epoch 9/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0406 - accuracy: 0.9846 - val_loss: 0.4518 - val_accuracy: 0.9088
Epoch 10/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0299 - accuracy: 0.9890 - val_loss: 0.4437 - val_ac